# 要約 
このJupyter Notebookは、Kaggleの「20の質問」ゲームにおける質問者と回答者エージェントのテストを行うために設計されています。Kaggle環境のメモリ制限のため、ローカル環境でのテストが主な目的です。ノートブック内では、エージェントが同ゲームにおいて戦略的に質問を行い、回答を生成するプロセスをシミュレートしています。

### 取り組む問題
このノートブックは、言語モデルを活用して、対話式の情報収集ゲームである「20の質問」において、エージェントのパフォーマンスを評価することを目的としています。具体的に、質問者が有効な質問を行い、回答者が適切に応答することで、ターゲットワードを特定する能力が試されます。

### 使用されている手法とライブラリ
- **ライブラリ**:
  - `transformers`: Hugging Faceのライブラリを使用して、言語モデルを扱っています。
  - `torch`: PyTorchライブラリを利用し、データの操作とGPUでの推論をサポートしています。
  - `kaggle_secrets`: Kaggle内でのシークレット管理に対応しています。
  - `pandas`と`json`: キーワードデータの処理及び構造化を行うために使用されています。

- **主要な手法**:
  - **トークナイゼーション**: `AutoTokenizer`を使用して、入力テキストをトークンに変換し、モデルが処理できる形式にしています。
  - **モデルの生成**: `AutoModelForCausalLM`モデルを用いて、質問や推測に対する回答を生成します。
  - **ゲームロジック**: `asker`や`answerer`メソッドを通じて、エージェントの質問者および回答者としての動作を定義し、対話を進めるロジックが実装されています。

このノートブックは、作成したエージェントが「20の質問」を通じて、適切な質問や推測を行う能力をテストするために、キーワード情報を基にしたサンプルラウンドを実行します。また、各ラウンドではエージェントの出力をモニタリングし、勝利条件をチェックする仕組みも組み込まれています。

---


# 用語概説 
以下は、ノートブック内で機械学習や深層学習の初心者がつまずきがちな専門用語についての簡単な解説です。特に、実務を経験していないと馴染みのない用語に焦点を当てています。

1. **CUDA（Compute Unified Device Architecture）**
   - NVIDIAが開発した並列計算のためのプラットフォームおよびAPIです。深層学習モデルのトレーニングや推論を高速化するために、GPUを使って計算を行うことができます。

2. **トークナイザー（Tokenizer）**
   - 自然言語処理（NLP）において、文を意味を持つ単位（トークン）に分解するプロセスです。たとえば、文章を単語や文字列に分けることによって、モデルがテキストデータを理解しやすくします。

3. **Causal Language Model（因果言語モデル）**
   - 文脈を考慮して、次に来る単語を生成するために設計されたモデルです。このモデルは、過去の単語をもとに次の単語を予測するため、因果関係を持っています。

4. **デバイスマップ（Device Map）**
   - モデルがどのハードウェアデバイス（CPUやGPU）上で動作するかを管理する仕組みです。特に大規模なモデルの場合、メモリ効率を考慮して複数のデバイスに分割して動かすことがあります。

5. **終了トークン（End of Text Token, EOT）**
   - テキスト生成モデルにおいて、生成が終了したことを示す特別なトークンです。このトークンが出力されると、モデルは生成を停止します。

6. **推測（Guessing）**
   - 20の質問ゲームにおいて、質問者がある単語を特定しようとする行為です。「推測する」という行為は、モデルが過去の質問に基づいて、元のキーワードを推測することを意味します。

7. **観察クラス（Observation Class）**
   - ゲームの状態を管理するためのクラスです。質問や回答、推測の情報を保持し、ゲームの進行に必要なデータを提供します。

8. **エージェント（Agent）**
   - 自律的に環境で行動するプログラムのことです。このコンテキストでは、質問者または回答者の役割を果たすAIが該当します。

9. **メモリ効率的なSDP（Symmetric Distributed Processing）**
   - メモリを節約しながら計算を効率的に行うためのCUDAの機能です。この機能を使用することで、GPUのメモリ使用量を削減できます。

10. **代替キーワード（Alternative Keywords）**
    - ゲーム内で使用できる別のキーワードのリストです。主なキーワードに関連する他の用語を提供し、推測を助ける役割を果たします。

これらの用語は、特にJupyter Notebook内で特有の機能や概念に関連しているため、初心者が理解するためにはさらに詳しい説明が必要です。

---


# Kaggle環境を使用せずにエージェントをテストできます
kaggle_environmentsを使用したテストは、メモリ制限のために実施できません。  
そのため、質問者と回答者エージェントをテストできるコードを作成しました。

このエージェントは、以下に基づいて作成されました：  
[llama3-8b- LLM20 Questions [LB 750]](https://www.kaggle.com/code/wouldyoujustfocus/llama3-8b-llm20-questions-lb-750)

In [ ]:
%%bash
# /kaggle/working/submission ディレクトリを作成します
# -p オプションを使用することで、親ディレクトリが存在しない場合でも、必要なディレクトリをすべて作成します
mkdir -p /kaggle/working/submission

In [ ]:
%%writefile -a submission/main.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from kaggle_secrets import UserSecretsClient
import os
import sys
import shutil

# CUDAのメモリ効率的なSDPを無効にします
torch.backends.cuda.enable_mem_efficient_sdp(False)
# CUDAのフラッシュSDPを無効にします
torch.backends.cuda.enable_flash_sdp(False)

# エージェントのモデルパスを設定します
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")
else:
    model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

# トークナイザーを初期化します
tokenizer = AutoTokenizer.from_pretrained(model_id)
# モデルを初期化します
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
# 終了トークンのIDを取得します
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]


def generate_answer(template):
    # 入力テンプレートをトークン化します
    inp_ids = tokenizer(template, return_tensors="pt").to("cuda")
    # モデルから回答を生成します
    out_ids = model.generate(**inp_ids,max_new_tokens=15).squeeze()
    # 生成結果の開始位置を特定します
    start_gen = inp_ids.input_ids.shape[1]
    out_ids = out_ids[start_gen:]
    # 終了トークンが生成結果に含まれているか確認します
    if id_eot in out_ids:
        stop = out_ids.tolist().index(id_eot)
        out = tokenizer.decode(out_ids[:stop])
    else:
        out = tokenizer.decode(out_ids)
    return out
    

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        # モードが適切であることを確認します
        assert mode in ["asking", "guessing", "answering"], "modeは以下のいずれかの値でなければなりません: asking, answering, guessing"
        if mode == "asking":
            # 質問者の役割を開始します
            output = self.asker(obs)
        if mode == "answering":
            # 回答者の役割を開始します
            output = self.answerer(obs)
            # "yes"または"no"の応答を標準化します
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "yes"
        if mode == "guessing":
            # 推測する役割を開始します
            output = self.asker(obs)
        return output
    
    
    def asker(self, obs):
        sys_prompt = """
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者の「はい」または「いいえ」で答えます。
        キーワードは特定の場所、または物です。\n
        """
    
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            ヒントとして、キーワードがモロッコである場合の動作例を示します:
            例:
            <あなた: それは場所ですか？
            ユーザー: はい
            あなた: それはヨーロッパにありますか？
            ユーザー: いいえ
            あなた: それはアフリカにありますか？
            ユーザー: はい
            あなた: そこに住んでいる人々の大半は肌の色が濃いですか？
            ユーザー: いいえ
            あなた: それはmで始まる国名ですか？
            ユーザー: はい
            あなた: それはモロッコですか？
            ユーザー: はい。>

            ユーザーが単語を選びました。最初の質問をしてください！
            短くて簡潔に、1つの質問だけを与えてください。余分な言葉はいりません！"""
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{ask_prompt}<|eot_id|>"""
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
            if len(obs.questions)>=1:
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"{q}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                    chat_template += f"{a}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問: {q}\n回答: {a}\n"""
            guess_prompt =  sys_prompt + f"""
            現在のゲームの状態は以下の通りです:\n{conv}
            会話に基づいて単語を推測できますか？単語だけを答えてください。余計なことはいりません。
            """
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{guess_prompt}<|eot_id|>"""
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
                
        output = generate_answer(chat_template)        
        return output
        
        
        
    def answerer(self, obs):
        sys_prompt = f"""
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者の「はい」または「いいえ」で答えます。
        キーワードは特定の場所、または物です。\n
        ユーザーの質問を理解し、プレイしているキーワードを理解することを確認してください。
        現在、ユーザーが推測すべき単語は: "{obs.keyword}" で、カテゴリは "{obs.category}" です。
        ヒントとして、キーワードがモロッコで、カテゴリが「場所」である場合の動作例を示します:
        例:
        <ユーザー: それは場所ですか？
        あなた: はい
        ユーザー: それはヨーロッパにありますか？
        あなた: いいえ
        ユーザー: それはアフリカにありますか？
        あなた: はい
        ユーザー: そこに住んでいる人々の大半は肌の色が濃いですか？
        あなた: いいえ
        ユーザー: それはmで始まる国名ですか？
        あなた: はい
        ユーザー: それはモロッコですか？
        あなた: はい。>
        """
        
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{sys_prompt}<|eot_id|>"""
        chat_template += "<|start_header_id|>user<|end_header_id|>\n\n"
        chat_template += f"{obs.questions[0]}<|eot_id|>"
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{a}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                chat_template += f"{q}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        output = generate_answer(chat_template)
        return output
    
    
robot = Robot()


def agent(obs, cfg):
    
    if obs.turnType =="ask":
        # 質問の際の応答を生成します
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        # 推測の際の応答を生成します
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        # 回答の際の応答を生成します
        response = robot.on(mode = "answering", obs = obs)
        
    # 応答がない場合、デフォルトで「yes」を返します
    if response == None or len(response)<=1:
        response = "yes"
        
    return response

## 次のコードをmain.pyの後に貼り付けてください

In [ ]:
# keywords_local.pyという名前で、指定したURLからファイルをダウンロードします
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

# 観察クラスを定義します
class Observation:
    def __init__(self):
        self.step = 0  # 現在のステップ
        self.role = "guesser"  # 役割は推測者
        self.turnType = "ask"  # 現在のターンタイプは質問
        self.keyword = "Japan"  # 予め設定されたキーワード（例として「日本」）
        self.category = "country"  # キーワードのカテゴリ（例として「国」）
        self.questions = []  # 質問のリスト
        self.answers = []  # 回答のリスト
        self.guesses = []  # 推測のリスト
        
# KEYWORDS_JSONからキーワードデータフレームを作成する関数
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSON文字列を辞書に変換

    keyword_list = []  # キーワードのリスト
    category_list = []  # カテゴリのリスト
    alts_list = []  # 代替キーワードのリスト

    # JSONデータを反復処理してリストを作成
    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']  # キーワードを抽出
            keyword_list.append(keyword)  # リストに追加
            category_list.append(json_data[i]['category'])  # カテゴリをリストに追加
            alts_list.append(json_data[i]['words'][j]['alts'])  # 代替キーワードをリストに追加

    # データフレームを作成
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])  # 列名を指定
    data_pd['keyword'] = keyword_list  # キーワードリストをデータフレームに追加
    data_pd['category'] = category_list  # カテゴリリストをデータフレームに追加
    data_pd['alts'] = alts_list  # 代替リストをデータフレームに追加
    
    return data_pd  # 作成したデータフレームを返す
    
# KEYWORDS_JSONを使用してキーワードデータフレームを作成
keywords_df = create_keyword_df(KEYWORDS_JSON)

In [ ]:
# 観察インスタンスを作成します
obs = Observation()
cfg = "_"  # 構成用の任意の文字列を設定

# キーワードデータフレームからサンプルを取得します
sample_df = keywords_df.sample()
obs.keyword = sample_df["keyword"].values[0]  # サンプルのキーワードを観察オブジェクトに設定
obs.category = sample_df["category"].values[0]  # サンプルのカテゴリを観察オブジェクトに設定
alts_list = sample_df["alts"].values[0]  # サンプルの代替キーワードリストを取得
alts_list.append(obs.keyword)  # オリジナルのキーワードを代替リストに追加

# 現在のキーワードを表示します
print(f"キーワード: {obs.keyword}")

# 最大20ラウンド分のゲームを実行します
for round in range(20):
    obs.step = round + 1  # ステップを更新
    
    obs.role = "guesser"  # 役割を推測者に設定
    obs.turnType = "ask"  # ターンタイプを質問に設定
    question = agent(obs, cfg)  # 質問を生成
    obs.questions.append(question)  # 質問を観察オブジェクトに追加
    
    obs.role = "answerer"  # 役割を回答者に設定
    obs.turnType = "answer"  # ターンタイプを回答に設定
    answer = agent(obs, cfg)  # 回答を生成
    obs.answers.append(answer)  # 回答を観察オブジェクトに追加
    
    obs.role = "guesser"  # 役割を推測者に戻す
    obs.turnType = "guess"  # ターンタイプを推測に設定
    guess = agent(obs, cfg)  # 推測を生成
    obs.guesses.append(guess)  # 推測を観察オブジェクトに追加
    
    # 現在のラウンド情報を表示します
    print(f"ラウンド: {round + 1}")
    print(f"質問: {question}")
    print(f"回答: {answer}")
    print(f"推測: {guess}")
    
    # 推測が代替リストに含まれているか確認します
    if guess in alts_list:
        print("勝利！！")  # 勝利のメッセージを表示
        break